In [0]:
from pyspark import SparkContext
sc = SparkContext . getOrCreate ()

sqlDF = spark .sql(" SELECT count(*) FROM shotData ")
sqlDF . show ()

+--------+
|count(1)|
+--------+
| 4966816|
+--------+



In [0]:
%sql
-- Q3 - Which are the top 10 players who attempted the most 3 point shots? What percentage of the time do each of those players prefer Layup Shots, Jump Shots, Slam Dunk Shots, etc.?
SELECT *
FROM (SELECT ShotDist.PLAYER_NAME,
			 ShotDist.ACTION_TYPE,
			 ROUND(ShotDist.NUM_SHOTS*100/TotalShots.TOTAL_SHOTS, 4) AS PERCENT_OF_SHOTS
	  FROM (SELECT PLAYER_NAME,
				   COUNT(*) AS TOTAL_SHOTS
			FROM shotdata
			GROUP BY PLAYER_NAME) AS TotalShots INNER JOIN
		   (SELECT shotdata.PLAYER_NAME,
				   CASE WHEN UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1)) = ''
						THEN UPPER(shotdata.ACTION_TYPE)
						ELSE UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1))
				   END AS ACTION_TYPE,
				   COUNT(*) AS NUM_SHOTS
			FROM (SELECT PLAYER_NAME
				  FROM shotdata
				  WHERE SHOT_TYPE = '3PT Field Goal'
				  GROUP BY PLAYER_NAME
				  ORDER BY COUNT(*) DESC
				  LIMIT 10) AS 3ptPlayers INNER JOIN
				 shotdata ON 3ptPlayers.PLAYER_NAME = shotdata.PLAYER_NAME
			GROUP BY shotdata.PLAYER_NAME, CASE WHEN UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1)) = ''
												THEN UPPER(shotdata.ACTION_TYPE)
												ELSE UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1))
										   END
			ORDER BY shotdata.PLAYER_NAME, CASE WHEN UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1)) = ''
												THEN UPPER(shotdata.ACTION_TYPE)
												ELSE UPPER(RIGHT(shotdata.ACTION_TYPE, CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE), CHARINDEX(' ', REVERSE(shotdata.ACTION_TYPE)) + 1) - 1))
										   END) AS ShotDist ON TotalShots.PLAYER_NAME = ShotDist.PLAYER_NAME 
	  ORDER BY ShotDist.PLAYER_NAME, ShotDist.ACTION_TYPE)
PIVOT (MAX(PERCENT_OF_SHOTS)
	   FOR ACTION_TYPE IN ('BANK SHOT', 'DUNK SHOT', 'FADEAWAY SHOT', 'HOOK SHOT', 'JUMP SHOT', 'LAYUP SHOT', 'NO SHOT', 'TIP SHOT'))

PLAYER_NAME,BANK SHOT,DUNK SHOT,FADEAWAY SHOT,HOOK SHOT,JUMP SHOT,LAYUP SHOT,NO SHOT,TIP SHOT
James Harden,0.5112,2.2749,0.262,0.2748,68.7137,27.8037,0.032,0.1278
Kyle Korver,0.6361,0.2156,0.1401,0.2048,92.9711,5.6059,null,0.1725
Jamal Crawford,1.0878,0.6563,0.0729,0.3525,84.4728,12.9383,null,0.237
Damian Lillard,0.6796,1.5043,0.4276,0.252,69.1127,27.9627,0.0229,0.0382
Jason Terry,0.6352,0.7,0.013,0.2333,84.4309,13.7866,null,0.0519
Ray Allen,0.6789,2.1563,0.0489,0.1955,79.6915,16.7726,null,0.2716
Vince Carter,0.9841,5.1983,0.5749,1.3446,72.4544,16.9882,0.0049,1.6808
LeBron James,1.543,7.7424,1.9238,1.115,57.8681,28.9164,null,0.5536
Stephen Curry,1.791,0.2135,0.2204,0.2893,77.9913,19.4462,null,0.0482
Kobe Bryant,1.6452,4.279,1.5345,0.626,73.5896,17.3906,null,0.584


In [0]:
%sql
-- Q4 - What section of the court (as determined by SHOT_ZONE_BASIC and SHOT_ZONE_AREA) does each player shoot from? Where is each player most successful?

SELECT PlayerAttempts.PLAYER_NAME,
	   PlayerAttempts.MOST_ATTEMPTED_REGION,
	   PlayerFGPct.MOST_MADE_REGION
FROM (SELECT PlayerRegions2.PLAYER_NAME,
			 CONCAT(PlayerRegions2.SHOT_ZONE_AREA, ', ', PlayerRegions2.SHOT_ZONE_BASIC) AS MOST_ATTEMPTED_REGION,
			 PlayerRegions2.NUM_ATTEMPTS
	  FROM (SELECT PlayerRegions.PLAYER_NAME,
				   MAX(PlayerRegions.NUM_ATTEMPTS) AS MAX_ATTEMPTS
			FROM (SELECT PLAYER_NAME,
						 SHOT_ZONE_BASIC,
						 SHOT_ZONE_AREA,
						 COUNT(*) AS NUM_ATTEMPTS
				  FROM shotdata
				  GROUP BY PLAYER_NAME, SHOT_ZONE_BASIC, SHOT_ZONE_AREA
				  HAVING COUNT(*) > 30) AS PlayerRegions
			GROUP BY PlayerRegions.PLAYER_NAME) AS MaxAttempts INNER JOIN
		   (SELECT PLAYER_NAME,
				   SHOT_ZONE_BASIC,
				   SHOT_ZONE_AREA,
				   COUNT(*) AS NUM_ATTEMPTS
			FROM shotdata
			GROUP BY PLAYER_NAME, SHOT_ZONE_BASIC, SHOT_ZONE_AREA
			HAVING COUNT(*) > 30) AS PlayerRegions2 ON MaxAttempts.PLAYER_NAME = PlayerRegions2.PLAYER_NAME AND MaxAttempts.MAX_ATTEMPTS = PlayerRegions2.NUM_ATTEMPTS) AS PlayerAttempts INNER JOIN
	 (SELECT PlayerRegions2.PLAYER_NAME,
			 CONCAT(PlayerRegions2.SHOT_ZONE_AREA, ', ', PlayerRegions2.SHOT_ZONE_BASIC) AS MOST_MADE_REGION, 
			 PlayerRegions2.FG_PCT
	  FROM (SELECT PlayerRegions.PLAYER_NAME,
				   MAX(PlayerRegions.FG_PCT) AS MAX_FG_PCT
			FROM (SELECT PLAYER_NAME,
						 SHOT_ZONE_BASIC,
						 SHOT_ZONE_AREA,
						 ROUND(SUM(SHOT_MADE_FLAG)*100/COUNT(*), 2) AS FG_PCT
				  FROM shotdata
				  GROUP BY PLAYER_NAME, SHOT_ZONE_BASIC, SHOT_ZONE_AREA
				  HAVING COUNT(*) > 30) AS PlayerRegions
			GROUP BY PlayerRegions.PLAYER_NAME) AS MaxFGPct INNER JOIN
		   (SELECT PLAYER_NAME,
				   SHOT_ZONE_BASIC,
				   SHOT_ZONE_AREA,
				   ROUND(SUM(SHOT_MADE_FLAG)*100/COUNT(*), 2) AS FG_PCT
			FROM shotdata
			GROUP BY PLAYER_NAME, SHOT_ZONE_BASIC, SHOT_ZONE_AREA
			HAVING COUNT(*) > 30) AS PlayerRegions2 ON MaxFGPct.PLAYER_NAME = PlayerRegions2.PLAYER_NAME AND MaxFGPct.MAX_FG_PCT = PlayerRegions2.FG_PCT) AS PlayerFGPct ON PlayerAttempts.PLAYER_NAME = PlayerFGPct.PLAYER_NAME
ORDER BY PlayerAttempts.PLAYER_NAME

PLAYER_NAME,MOST_ATTEMPTED_REGION,MOST_MADE_REGION
A.C. Green,"Center(C), Restricted Area","Center(C), Restricted Area"
A.J. Guyton,"Right Side Center(RC), Above the Break 3","Center(C), Restricted Area"
AJ Price,"Right Side Center(RC), Above the Break 3","Center(C), Restricted Area"
Aaron Brooks,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron Gordon,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron Gray,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron Holiday,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron McKie,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron Nesmith,"Center(C), Restricted Area","Center(C), Restricted Area"
Aaron Wiggins,"Center(C), Restricted Area","Center(C), Restricted Area"


In [0]:
%sql
--Q7a - What is the average points per attempt for each player with a minimum of 300 attempts? Who has the highest average points per attempt during the last 5 minutes of a game (minimum 200 attempts)?

SELECT MAX(PLAYER_NAME) AS PLAYER_NAME,
       ROUND(AVG(NUM_POINTS), 2) AS AVG_PTS_PER_ATT 
FROM
(SELECT PLAYER_ID, PLAYER_NAME,
        CASE WHEN SHOT_TYPE = '2PT Field Goal' AND SHOT_MADE_FLAG = 1
             THEN 2
             WHEN SHOT_TYPE = '3PT Field Goal' AND SHOT_MADE_FLAG = 1
             THEN 3
             ELSE 0
             END AS NUM_POINTS
FROM shotdata) AS PlayerPoints
GROUP BY PLAYER_ID
HAVING COUNT(*) >= 300
ORDER BY AVG_PTS_PER_ATT DESC

PLAYER_NAME,AVG_PTS_PER_ATT
Robert Williams III,1.46
Mitchell Robinson,1.44
Daniel Gafford,1.38
DeAndre Jordan,1.35
Damian Jones,1.35
Onyeka Okongwu,1.34
Rudy Gobert,1.31
Nic Claxton,1.3
Jaxson Hayes,1.3
Tony Bradley,1.27


In [0]:
%sql
--Q7b - Who has the highest average points per attempt during the last 5 minutes of a game (minimum 200 attempts)?

SELECT MAX(PLAYER_NAME) AS PLAYER_NAME,
       ROUND(AVG(NUM_POINTS), 2) AS AVG_PTS_PER_ATT 
FROM
(SELECT PLAYER_ID, PLAYER_NAME,
        CASE WHEN SHOT_TYPE = '2PT Field Goal' AND SHOT_MADE_FLAG = 1
             THEN 2
             WHEN SHOT_TYPE = '3PT Field Goal' AND SHOT_MADE_FLAG = 1
             THEN 3
             ELSE 0
             END AS NUM_POINTS
FROM shotdata
WHERE PERIOD = 4 AND 60*MINUTES_REMAINING + SECONDS_REMAINING <= 300) AS PlayerPoints
GROUP BY PLAYER_ID
HAVING COUNT(*) >= 200
ORDER BY AVG_PTS_PER_ATT DESC

PLAYER_NAME,AVG_PTS_PER_ATT
Deandre Ayton,1.42
DeAndre Jordan,1.37
Rudy Gobert,1.36
Hassan Whiteside,1.34
Clint Capela,1.33
Montrezl Harrell,1.3
John Collins,1.28
Seth Curry,1.27
Mason Plumlee,1.24
Tyson Chandler,1.24


In [0]:
%sql
--Q5 What is the average shot distance per period?
Select avg(SHOT_DISTANCE) as avgShotDist, 
CASE 
  WHEN floor(GAME_DATE/10000) < 2000 THEN '1990s (late)' 
  WHEN floor(GAME_DATE/10000) < 2005 THEN '2000s (early)' 
  WHEN floor(GAME_DATE/10000) < 2010 THEN '2000s (late)' 
  WHEN floor(GAME_DATE/10000) < 2015 THEN '2010s (early)' 
  WHEN floor(GAME_DATE/10000) < 2020 THEN '2010s (late)' 
  ELSE '2020s' END
as era 
from shotdata 
group by era
order by era;

avgShotDist,era
10.795736939471919,1990s (late)
11.609678863129671,2000s (early)
11.917641387461853,2000s (late)
12.058440940209545,2010s (early)
12.861003508190173,2010s (late)
13.671008213077243,2020s


In [0]:
%sql
--Q6 What team has the highest field goal percentage for any team in any year? Which has the lowest?
Select Team_Name, floor(GAME_DATE/10000) as gameYear, 
(sum(SHOT_MADE_FLAG)/sum(SHOT_ATTEMPTED_FLAG))*100 as goal_percent
FROM shotdata
where SHOT_TYPE = "2PT Field Goal" OR SHOT_TYPE = "3PT Field Goal"
group BY TEAM_NAME, gameYear
ORDER BY goal_percent DESC

Team_Name,gameYear,goal_percent
Utah Jazz,1998,50.29893423446842
Phoenix Suns,2009,50.28058361391695
Phoenix Suns,2008,50.280548628428924
Miami Heat,2013,50.16483516483517
Golden State Warriors,2017,49.92513951272628
Denver Nuggets,2022,49.76280834914611
Utah Jazz,1997,49.55442393380013
Phoenix Suns,2007,49.31487498234214
Golden State Warriors,2016,49.258805929552565
Phoenix Suns,2022,49.226441631504926


In [0]:
%sql
-- Q1 Whether popular “regions” of the court to attempt shots for have shifted over time
With popularZones as (
  Select SHOT_ZONE_AREA, floor(GAME_DATE/10000) as gameYear, 
  count(*) as numShots
  FROM shotdata
  group BY SHOT_ZONE_AREA, gameYear
  ORDER BY numShots DESC
),
maxYearShots as (
  Select gameYear, max(numShots) as maxShots
  FROM popularZones 
  GROUP BY gameYear
)
SELECT popularZones.gameYear, popularZones.SHOT_ZONE_AREA, popularZones.numShots
FROM popularZones
INNER JOIN maxYearShots on maxYearShots.gameYear = popularZones.gameYear
Where popularZones.numShots = maxYearShots.maxShots
Order by popularZones.gameYear

gameYear,SHOT_ZONE_AREA,numShots
1996,Center(C),35091
1997,Center(C),99079
1998,Center(C),62302
1999,Center(C),92249
2000,Center(C),96524
2001,Center(C),93725
2002,Center(C),95914
2003,Center(C),95171
2004,Center(C),93431
2005,Center(C),99507


In [0]:
%sql

--Q2: What is the average field goal percentage for home teams vs away teams? How   has this changed over time?
With checkedIfHome as (
  Select shotdata.*, 
  (nbateams.ABBREVIATION = shotdata.HTM) as wasHomeTeam,
  SHOT_ZONE_AREA, floor(GAME_DATE/10000) as gameYear from
  shotdata
  inner join nbateams on nbateams.TEAM_ID = shotdata.TEAM_ID
  where SHOT_TYPE = "2PT Field Goal" OR SHOT_TYPE = "3PT Field Goal"
)
Select Distinct wasHomeTeam, gameYear,
(sum(SHOT_MADE_FLAG)/sum(SHOT_ATTEMPTED_FLAG))*100 as goal_percent
from checkedIfHome
group by wasHomeTeam, gameYear
Order by gameYear, wasHomeTeam


wasHomeTeam,gameYear,goal_percent
false,1996,44.34347016579484
true,1996,45.34549220413095
false,1997,45.19641743228778
true,1997,45.828971778581575
false,1998,45.03209211468229
true,1998,45.667653412272976
false,1999,43.41785843047711
true,1999,44.523593708344436
false,2000,43.972430795132546
true,2000,45.36239532405978
